In [4]:
from sklearn import datasets
# train and test set for cross-validation
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
X = iris.data
y = iris.target

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)


In [10]:
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file

dump_svmlight_file(X_train, y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

[22:07:48] 120x4 matrix with 480 entries loaded from dtrain.svm
[22:07:48] 30x4 matrix with 120 entries loaded from dtest.svm


In [17]:
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations


In [18]:
bst = xgb.train(param, dtrain, num_round)

In [19]:
bst.dump_model('dump.raw.txt')

In [20]:
preds = bst.predict(dtest)

In [26]:
preds[1:10,:]

array([[0.98254657, 0.01395846, 0.00349498],
       [0.00363749, 0.00615226, 0.9902103 ],
       [0.00564738, 0.97917044, 0.0151822 ],
       [0.00540075, 0.93640935, 0.0581899 ],
       [0.98607963, 0.0104128 , 0.00350755],
       [0.00438964, 0.99041265, 0.0051977 ],
       [0.0156953 , 0.06653062, 0.917774  ],
       [0.0063378 , 0.94877166, 0.04489058],
       [0.00438964, 0.99041265, 0.0051977 ]], dtype=float32)

- Here each column represents class number 0, 1, or 2. For each line you need to select that column where the probability is the highest:

In [27]:
import numpy as np
best_preds = np.asarray([np.argmax(line) for line in preds])

- Now you get a nice list with predicted classes:

In [28]:
best_preds

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

- Determine the precision of this prediction:

In [33]:
from sklearn.metrics import precision_score
print(precision_score(y_test, best_preds, average='macro'))

1.0
